# Baobab model project

**Note: interactive plots in this notebook are not visible in Github. To view, download and run this notebook in the Jupyter Notebook environment.**

Passive investment funds are becoming popular, however the companies that are included in these schemes are often the subject of controversy, engage in dubious business practices or create products that have a nett harmful effect on the world. The Babab eithical fund aims to provides a low-cost investment scheme that allows investors to avoid funding companies deemed harmful or unethical. Essentially, the fund consists of the companies listed on the JSE, with unsuitable companies excluded. This model aims to determine if the Baobab ethical fund would outperform the Johannesburg Stock Exchange (or a pared-down version of it) over a period of 5 financial years. 

A list of JSE-listed companies is imported. Some companies are removed due to missing historical data. The Baobab ethical fund is compared to these companies, as well as the JSE as a whole to confirm that the missing company data doesn't affect the prediction. When making the comparisons between the pseudo-JSE and the Baobab fund, each is modeled as an equal-weighted fund in the absense of information about numbers of shares available for each company. The actual JSE all-share price, however, takes the number into account. As such, the pseudo-JSE (calculated as an equal fund) and actual JSE all-share price are shown on each chart for comparison.

Baobab is constructed by starting with the pseudo-JSE model (350 companies) and removing a further 60 companies that are deemed to not meet the ethical criteria for inclusion. 

At each day, the price of each (the JSE and Baobab) are obtained by summing the value of stocks registered with each. Then two simulated buying scenarios are created:
* Dollar-cost averaging: 30x R10,000.00 =  R300k investment
* Single-shot R300k innvestment on Day 1

The performance of these is analyzed over an approximate 5 year period, starting in 2013, and plotted against one another.

## Initial imports and setup:

In [1]:
# DataFrames and importing:
import pandas as pd
import numpy as np
import pandas_datareader.data as web

#Plotting:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import matplotlib.pyplot as plt
%matplotlib inline

init_notebook_mode(connected=True)

#other
import datetime

## Read in data:

### Supplied company inclusion list, etc:
The list of "ethical" companies was supplied in the file "companyInfo.csv". It is read in here:

In [2]:
#read in data
AllCompanyInfo = pd.read_csv('companyInfo.csv') 

#Pull JSE code column, and use as index
AllCompanyInfo.index = AllCompanyInfo['JSE code'] 

## Import actual JSE price data for use as a validation metric

In [3]:
#read JSE data from CSV:
JSE_actual = pd.read_csv("JSE_actual.csv")

# Convert date column to datetime object
DateCol = []
Months = "Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec".split()
for dateString in JSE_actual["Date"]:
    Month = Months.index(dateString.replace(',', ' ').split()[0])+1
    Day = int(dateString.replace(',', ' ').split()[1])
    Year = int(dateString.replace(',', ' ').split()[2])
    DateCol.append(datetime.datetime(Year, Month, Day))
JSE_actual.index = pd.to_datetime(JSE_actual['Date'])

# Remove unnecessary columns and sort ascending
JSE_actual.drop("Date Open High Low Vol percent".split(), axis = 1, inplace = True)
JSE_actual = JSE_actual.sort_index()

### Historical market data from the client
Historical JSE data is only available for companies currently listed on the JSE. The client provided a spreadsheet of this info. It is read in here.

Note that there is missing data - some companies appear in the "companyInfo.csv" spreadsheet that are not in Georgina's list, and vice versa. Any company that does not appear in both is excluded. This leaves us with a totoal of 350 companies in the the pseudo-JSE.

In [4]:
AllStocks = pd.read_csv('close_prices.csv', index_col='date')
AllStocks.drop(columns = 'Unnamed: 0', axis = 1, inplace=True)

# Convert column headings to appropriate ticker symbols
AllStocks.columns = [x.replace('.Close', '').replace('.', ':')
                     for x in list(AllStocks.columns)]

# Convert date column from string to pandas datetime format
AllStocks.index = pd.to_datetime(AllStocks.index)

# Remove companies that are not in both lists:
for company in list(AllStocks.columns):
    if company not in list(AllCompanyInfo.index):
        AllStocks.drop(columns = company, axis = 1, inplace = True)
for company in list(AllCompanyInfo.index):
    if company not in list(AllStocks.columns):
        AllCompanyInfo.drop(index = company, axis = 0, inplace = True)
        
# Check:
if len(AllStocks.columns) != len(AllCompanyInfo.index):
    print('CAUTION: Mismatch in number of entries in data matrices')
else:
    print('Total companies included in pseudo-JSE list: '
          + str(len(AllStocks.columns)))

AllStocks = AllStocks.iloc[42:]    

Total companies included in pseudo-JSE list: 350


### Deal with elements in the JSE data that are missing

Below, we compare the dates in the JSE_actual dataframe with the Allstocks dataframe, to look for missing entries. If found, the missing entries are noted, and filled with a value of 1 to avoid divide-by-zero errors later.

In [5]:
for date in AllStocks.index:
    if date not in JSE_actual.index:
        JSE_actual.loc[date] = 1
        print("Missing Date: ", date)

if (len(AllStocks) == len(JSE_actual)):
    print("\nSuccess - both JSE and individual stock dataframes now have the same number of entries")
else:
    print("\nNOTE: Some dates are missing from the JSE data!")

Missing Date:  2015-08-24 00:00:00
Missing Date:  2015-08-25 00:00:00
Missing Date:  2015-08-26 00:00:00
Missing Date:  2015-08-27 00:00:00
Missing Date:  2015-08-28 00:00:00
Missing Date:  2015-08-31 00:00:00
Missing Date:  2015-09-01 00:00:00
Missing Date:  2015-09-02 00:00:00
Missing Date:  2015-09-03 00:00:00
Missing Date:  2015-09-04 00:00:00
Missing Date:  2015-09-07 00:00:00
Missing Date:  2015-09-08 00:00:00
Missing Date:  2015-09-09 00:00:00

Success - both JSE and individual stock dataframes now have the same number of entries


## Build list of ticker symbols of included companies:

The "comapanyInfo.csv" spreadsheet is used to build Baobab inclusion/exclusion lists.

In [6]:
IncludedCompanies = list(AllCompanyInfo[AllCompanyInfo['No code'] == 'Yes'].index)
ExcludedCompanies = list(AllCompanyInfo[AllCompanyInfo['No code'] != 'Yes'].index)
print('Total companies included in Baobab list: ' + str(len(IncludedCompanies)))
print('Companies excluded: ' + str(len(ExcludedCompanies)))

Total companies included in Baobab list: 290
Companies excluded: 60


### JSE and Baobab prices

Below we plot the total price to purchase one of each stock in each fund (i.e one of each stock in Baobab and one of each in the pseudo-JSE).

In [7]:
import plotly.graph_objs as go
Price = pd.DataFrame() #Empty dataframe
Price['JSE_calculated'] = AllStocks.sum(axis = 1) 
Price['JSE_actual'] = JSE_actual
Price['Baobab'] = AllStocks[IncludedCompanies].sum(axis = 1)

trace0 = go.Scatter(
    x=Price.index,
    y=Price['JSE_calculated'],
    name='Pseudo-JSE (equal-weighted)'
)

trace1 = go.Scatter(
    x=Price.index,
    y=Price['JSE_actual'],
    name='Actual JSE All-Share Price'
)

trace2 = go.Scatter(
    x=Price.index,
    y=Price['Baobab'],
    name='Baobab (equal-weighted)'
)

iplot(go.Figure(data=[trace0, trace1, trace2]))

In the above chart, the missing JSE data is visible as a small discontinuity in the blue trace (actual JSE all-share price).

## Simulate purchasing of units

We consider two models: either investing R10,000 at the beginning of each month for 30 months (dollar-cost averaging), or investing R300k all at once on day 1 (single-shot investing). Both models result in a total of R300k in investment.

### Dollar cost averaging, purchasing R10,000 at the beginning of each month (excl public holidays)

In [8]:
# Dollar-cost averaging on 1st day of every month:

#Empty dataframe:
purchaseHistory = pd.DataFrame(np.zeros(shape = (len(AllStocks), 3)), 
                               columns = 'JSE_calculated JSE_actual Baobab'.split(), 
                               index = AllStocks.index)

totalInvestment = 0 #initialize
#simulate buying:
for date in list(purchaseHistory.index):
    if (date.is_month_start) or (date.dayofweek == 0 and date.day <=3) and (totalInvestment <= 290000):
        totalInvestment += 10000
        purchaseHistory['JSE_calculated'].loc[date] = 10000/Price['JSE_calculated'].loc[date] #buy at price on that day        
        if Price['JSE_actual'].loc[date] != 1:
            purchaseHistory['JSE_actual'].loc[date] = 10000/Price['JSE_actual'].loc[date] #buy at price on that day        
        purchaseHistory['Baobab'].loc[date] = 10000/Price['Baobab'].loc[date]  #buy at price on that day

In [9]:
# Held units
heldUnits = pd.DataFrame(np.zeros(shape = (len(AllStocks), 3)), 
                         columns = 'JSE_calculated JSE_actual Baobab'.split(), 
                         index = AllStocks.index)

# Day-one unit holdings:
heldUnits['JSE_calculated'].iloc[0] = purchaseHistory['JSE_calculated'].iloc[0]
heldUnits['JSE_actual'].iloc[0] = purchaseHistory['JSE_actual'].iloc[0]
heldUnits['Baobab'].iloc[0] = purchaseHistory['Baobab'].iloc[0]

#Holdings growth over time:
for date in range(1, len(heldUnits.index)): #ignore first entry (included manually above)
    #add new purchase to previous total
    heldUnits.iloc[date] = heldUnits.iloc[date-1] + purchaseHistory.iloc[date] 

Note: In the above dollar-cost averaging model, purchases on weekends are included but public holidays are skipped. If necessary, the model can be adjusted to incorporate public holiday purchases, but since this affects both funds the bias introduced is minimal.

### Plotting of the value of the holdings over time:

In [10]:
holdingsValue = pd.DataFrame(np.zeros(shape = (len(AllStocks), 3)), 
                             columns = 'JSE_calculated JSE_actual Baobab'.split(), 
                             index = AllStocks.index)
for date in list(holdingsValue.index):
    holdingsValue['JSE_calculated'].loc[date] = Price['JSE_calculated'].loc[date] * heldUnits['JSE_calculated'].loc[date]
    holdingsValue['JSE_actual'].loc[date] = Price['JSE_actual'].loc[date] * heldUnits['JSE_actual'].loc[date]
    holdingsValue['Baobab'].loc[date] = Price['Baobab'].loc[date]*heldUnits['Baobab'].loc[date]

trace0 = go.Scatter(
    x=holdingsValue.index,
    y=holdingsValue['JSE_calculated'],
    name='Pseudo-JSE (equal-weighted)'
)

trace1 = go.Scatter(
    x=holdingsValue.index,
    y=holdingsValue['JSE_actual'],
    name='Actual JSE All-Share Price'
)

trace2 = go.Scatter(
    x=holdingsValue.index,
    y=holdingsValue['Baobab'],
    name='Baobab (equal-weighted)'
)

iplot(go.Figure(data=[trace0, trace1, trace2]))

As for previous visualizations, the missing data in the JSE All-share price is visible as the discontinuity in the orange trace.

### Single purchace of R300,000 at the beginning of simulation

In [11]:
purchaseHistory = pd.DataFrame(np.zeros(shape = (len(AllStocks), 3)), columns = 'JSE_calculated JSE_actual Baobab'.split(), index = AllStocks.index)
purchaseHistory['JSE_calculated'].iloc[0] = 300000/Price['JSE_calculated'].iloc[0]
purchaseHistory['JSE_actual'].iloc[0] = 300000/Price['JSE_actual'].iloc[0]
purchaseHistory['Baobab'].iloc[0] = 300000/Price['Baobab'].iloc[0]

In [12]:
# Held units
heldUnits = pd.DataFrame(np.zeros(shape = (len(AllStocks), 3)), columns = 'JSE_calculated JSE_actual Baobab'.split(), index = AllStocks.index)
# Day-one unit holdings:
heldUnits['JSE_calculated'].iloc[0] = purchaseHistory['JSE_calculated'].iloc[0]
heldUnits['JSE_actual'].iloc[0] = purchaseHistory['JSE_actual'].iloc[0]
heldUnits['Baobab'].iloc[0] = purchaseHistory['Baobab'].iloc[0]

for date in range(1, len(heldUnits.index)): #ignore first entry (incuded manually above)
    heldUnits['JSE_calculated'].iloc[date] = heldUnits['JSE_calculated'].iloc[date-1] + purchaseHistory['JSE_calculated'].iloc[date]
    heldUnits['JSE_actual'].iloc[date] = heldUnits['JSE_actual'].iloc[date-1] + purchaseHistory['JSE_actual'].iloc[date]
    heldUnits['Baobab'].iloc[date] = heldUnits['Baobab'].iloc[date-1] + purchaseHistory['Baobab'].iloc[date]

In [13]:
holdingsValue = pd.DataFrame(np.zeros(shape = (len(AllStocks), 3)), columns = 'JSE_calculated JSE_actual Baobab'.split(), index = AllStocks.index)
for date in list(holdingsValue.index):
    holdingsValue['JSE_calculated'].loc[date] = Price['JSE_calculated'].loc[date] * heldUnits['JSE_calculated'].loc[date]
    holdingsValue['JSE_actual'].loc[date] = Price['JSE_actual'].loc[date] * heldUnits['JSE_actual'].loc[date]
    holdingsValue['Baobab'].loc[date] = Price['Baobab'].loc[date]*heldUnits['Baobab'].loc[date]

trace0 = go.Scatter(
    x=holdingsValue.index,
    y=holdingsValue['JSE_calculated'],
    name='Pseudo-JSE (equal-weighted)'
)

trace1 = go.Scatter(
    x=holdingsValue.index,
    y=holdingsValue['JSE_actual'],
    name='Actual JSE All-Share Price'
)

trace2 = go.Scatter(
    x=holdingsValue.index,
    y=holdingsValue['Baobab'],
    name='Baobab (equal-weighted)'
)

iplot(go.Figure(data=[trace0, trace1, trace2]))

## Model validation: Check that investing in excluded companies would not outperform the JSE:

Since the Baobab model seems to out-perform the pseudo-JSE, it stands to reason that an "anti-Baobab" fund (including only unethical companies) would be out-performed. We simulate this below by comparing the pseudo-JSE to the excluded stocks only, and only using the single-shot investing model:

### Price of excluded stocks:

In [14]:
Price = pd.DataFrame() #Empty dataframe
Price['JSE_calculated'] = AllStocks.sum(axis = 1)
Price['Excluded stocks'] = AllStocks[ExcludedCompanies].sum(axis = 1)
Price[['JSE_calculated', 'Excluded stocks']]

trace0 = go.Scatter(
    x=Price.index,
    y=Price['JSE_calculated'],
    name='Pseudo-JSE (equal-weighted)'
)

trace1 = go.Scatter(
    x=Price.index,
    y=Price['Excluded stocks'],
    name='Excluded stocks'
)

iplot(go.Figure(data=[trace0, trace1]))

In [15]:
purchaseHistory = pd.DataFrame(np.zeros(shape = (len(AllStocks), 2)), columns = ['JSE_calculated', 'Excluded stocks'], index = AllStocks.index)
purchaseHistory['JSE_calculated'].iloc[0] = 300000/Price['JSE_calculated'].iloc[0]
purchaseHistory['Excluded stocks'].iloc[0] = 300000/Price['Excluded stocks'].iloc[0]

In [16]:
# Held units
heldUnits = pd.DataFrame(np.zeros(shape = (len(AllStocks), 2)), columns = ['JSE_calculated', 'Excluded stocks'], index = AllStocks.index)
# Day-one unit holdings:
heldUnits['JSE_calculated'].iloc[0] = purchaseHistory['JSE_calculated'].iloc[0]
heldUnits['Excluded stocks'].iloc[0] = purchaseHistory['Excluded stocks'].iloc[0]

for date in range(1, len(heldUnits.index)): #ignore first entry (incuded manually above)
    heldUnits['JSE_calculated'].iloc[date] = heldUnits['JSE_calculated'].iloc[date-1] + purchaseHistory['JSE_calculated'].iloc[date]
    heldUnits['Excluded stocks'].iloc[date] = heldUnits['Excluded stocks'].iloc[date-1] + purchaseHistory['Excluded stocks'].iloc[date]

In [17]:
holdingsValue = pd.DataFrame(np.zeros(shape = (len(AllStocks), 2)), columns = ['JSE_calculated', 'Excluded stocks'], index = AllStocks.index)
for date in list(holdingsValue.index):
    holdingsValue['JSE_calculated'].loc[date] = Price['JSE_calculated'].loc[date] * heldUnits['JSE_calculated'].loc[date]
    holdingsValue['Excluded stocks'].loc[date] = Price['Excluded stocks'].loc[date]*heldUnits['Excluded stocks'].loc[date]

trace0 = go.Scatter(
    x=holdingsValue.index,
    y=holdingsValue['JSE_calculated'],
    name='Pseudo-JSE (equal-weighted)'
)

trace1 = go.Scatter(
    x=holdingsValue.index,
    y=holdingsValue['Excluded stocks'],
    name='Excluded stocks'
)

iplot(go.Figure(data=[trace0, trace1]))

Conclusion: The anti-Baobab is out-performed by the pseudo-JSE, which backs-up the validity of the model.